# Подключение к API. Открытые и закрытые


API - это инструмент связи с данными. Вы делаете запросы, API - возвращает ответ.
Бывают 2 типа API:
- __открытые__ - общедоступные
- __закрытые__ - требующие различных процедур авторизации (токен)

# Задача
Получить данные восхода и заката из __открытого__ API https://sunrise-sunset.org/ для своей геолокации

In [29]:
# импортируем библиотеку, которая позволяет получает по названию - координаты
import geocoder
import pandas as pd

In [17]:
city = geocoder.arcgis('Суздаль')
print(city.json)

{'address': 'Суздаль, Суздальский район, Владимирская область', 'bbox': {'northeast': [56.45495, 40.47544], 'southwest': [56.39695, 40.41744]}, 'confidence': 6, 'lat': 56.42595000000006, 'lng': 40.44644000000005, 'ok': True, 'quality': 'Locality', 'raw': {'name': 'Суздаль, Суздальский район, Владимирская область', 'extent': {'xmin': 40.41744, 'ymin': 56.39695, 'xmax': 40.47544, 'ymax': 56.45495}, 'feature': {'geometry': {'x': 40.44644000000005, 'y': 56.42595000000006}, 'attributes': {'Score': 100, 'Addr_Type': 'Locality'}}}, 'score': 100, 'status': 'OK'}


In [23]:
# запишем широту и долготу
coord = city.json['lat'], city.json['lng']
coord

(56.42595000000006, 40.44644000000005)

In [24]:
# библиотека для запросов к API
import requests

URL = 'https://api.sunrise-sunset.org/json'
params = {
    'lat': coord[0], # берем первый элемент из списка
    'lng': coord[1], # берем второй элемент из списка
    'date': '2023-10-11'
}

sun_r_s = requests.get(URL, params=params) 

sun_r_s

<Response [200]>

In [25]:
# смотрим, что вывел API с помощью метода .json
sun_r_s.json()

{'results': {'sunrise': '3:39:28 AM',
  'sunset': '2:30:36 PM',
  'solar_noon': '9:05:02 AM',
  'day_length': '10:51:08',
  'civil_twilight_begin': '3:03:43 AM',
  'civil_twilight_end': '3:06:21 PM',
  'nautical_twilight_begin': '2:20:16 AM',
  'nautical_twilight_end': '3:49:48 PM',
  'astronomical_twilight_begin': '1:36:07 AM',
  'astronomical_twilight_end': '4:33:57 PM'},
 'status': 'OK'}

In [28]:
# посмотрим на время восхода
sun_r_s.json()['results']['sunrise']

'3:39:28 AM'

In [33]:
# можно преобразовать эти данные из словаря в Dataframe
pd.DataFrame([sun_r_s.json()['results']])

,sunrise,sunset,solar_noon,day_length,civil_twilight_begin,civil_twilight_end,nautical_twilight_begin,nautical_twilight_end,astronomical_twilight_begin,astronomical_twilight_end
0,3:39:28 AM,2:30:36 PM,9:05:02 AM,10:51:08,3:03:43 AM,3:06:21 PM,2:20:16 AM,3:49:48 PM,1:36:07 AM,4:33:57 PM


# Задача

 Выгрузить маркетологам для анализа 50 самых популярных постов за все время существования сообщества Skyeng во ВКонтакте. Метрику популярности будем считать как сумму лайков, репостов и комментариев к каждому посту. 

## Часть 1

__Получаем собственный `access_token` в ВК__

In [1]:
# читаем токен из файла my_own_token.py, куда мы сохранили access_token
from my_own_token import access_token as token 

## Часть 2

__Формируем запрос, который получает 100 записей со стены сообщества__

In [3]:
# метод, который мы используем
url = 'https://api.vk.com/method/wall.get'

# параметры, которые мы указываем согласно документации
params = {'domain':'skyeng',
          'count':100,
          'access_token': token,
          'v':'5.131'
        }
req = requests.get(url, params = params)
res = req.json()
print('Кол-во записей:', len(pd.DataFrame(res['response'])))

Кол-во записей: 100


In [4]:
# преобразуем в dataframe
pd.DataFrame(res['response']['items'])

,donut,comments,marked_as_ads,short_text_rate,hash,type,carousel_offset,attachments,date,from_id,id,is_favorite,likes,owner_id,post_source,post_type,reposts,text,views,edited
0,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,7r_VssrRO1adrWpGoTKQYfg0g8AU,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1697032513,-51890028,97094,False,"{'can_like': 1, 'count': 19, 'user_likes': 0, ...",-51890028,{'type': 'vk'},post,"{'count': 8, 'user_reposted': 0}","«Коллеги, нужен синк по лидам. Смэтчимся по сл...",{'count': 2500},NaN
1,{'is_donut': False},"{'can_post': 1, 'count': 3, 'groups_can_post':...",0,0.8,tyQUYUZB0ldV_E8P17HvuvpGtZfd,post,NaN,"[{'type': 'video', 'video': {'response_type': ...",1696936308,-51890028,97019,False,"{'can_like': 1, 'count': 31, 'user_likes': 0, ...",-51890028,{'type': 'vk'},post,"{'count': 2, 'user_reposted': 0}",,{'count': 5893},NaN
2,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,XRTTBZ1MMzIxfHQhRS4Ra7Eik7VR,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1696854134,-51890028,96976,False,"{'can_like': 1, 'count': 29, 'user_likes': 0, ...",-51890028,{'type': 'vk'},post,"{'count': 9, 'user_reposted': 0}","“I’ve got a ton of homework to do, so I better...",{'count': 4078},1.696856e+09
3,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,WA_mAhtoeV3EHjXioS3Z5Q89qLBx,post,NaN,"[{'type': 'video', 'video': {'response_type': ...",1696835541,-51890028,96972,False,"{'can_like': 1, 'count': 22, 'user_likes': 0, ...",-51890028,{'type': 'vk'},post,"{'count': 6, 'user_reposted': 0}",,{'count': 6695},NaN
4,{'is_donut': False},"{'can_post': 1, 'count': 2, 'groups_can_post':...",0,0.8,_Dh35ZbNBcLADpNXORfLH4k_hUFV,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1696755420,-51890028,96970,False,"{'can_like': 1, 'count': 26, 'user_likes': 0, ...",-51890028,{'type': 'vk'},post,"{'count': 6, 'user_reposted': 0}",Мыслить нужно позитивно и видеть за окном не г...,{'count': 4669},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,{'is_donut': False},"{'can_post': 1, 'count': 2, 'groups_can_post':...",0,0.8,NIwzF6hNBPIweicqG2Z2IW7vQg1T,post,NaN,"[{'type': 'video', 'video': {'response_type': ...",1691074292,-51890028,93362,False,"{'can_like': 1, 'count': 75, 'user_likes': 0, ...",-51890028,{'type': 'vk'},post,"{'count': 6, 'user_reposted': 0}",,{'count': 5253},NaN
96,{'is_donut': False},"{'can_post': 1, 'count': 2, 'groups_can_post':...",0,0.8,w-HDzg4HEroS8qJY-CYF0wIYn_Gy,post,0.0,"[{'type': 'video', 'video': {'response_type': ...",1691068440,-51890028,93361,False,"{'can_like': 1, 'count': 23, 'user_likes': 0, ...",-51890028,{'type': 'vk'},post,"{'count': 2, 'user_reposted': 0}",Коммуналка - часть 2😎\n\nСегодня вместе с Джор...,{'count': 3730},NaN
97,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,ulV3ldAOW0h3EXXCWb9IIt6pnl5P,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1691052600,-51890028,93360,False,"{'can_like': 1, 'count': 21, 'user_likes': 0, ...",-51890028,{'type': 'vk'},post,"{'count': 9, 'user_reposted': 0}","Конечно, в любое время года можно хорошо прове...",{'count': 8753},NaN
98,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,jpEPKYz_divaBN2hZZ5dH7qaGhlp,post,0.0,"[{'type': 'video', 'video': {'response_type': ...",1690994880,-51890028,93357,False,"{'can_like': 1, 'count': 40, 'user_likes': 0, ...",-51890028,{'type': 'vk'},post,"{'count': 7, 'user_reposted': 0}","💥What’s cooking, good-looking?\n\nВсем привет,...",{'count': 7359},NaN


## Часть 3

__Выгружаем все записи со стены__

In [5]:
import time

post_count = res['response']['count']
posts = pd.DataFrame(res['response']['items'])
off = 100
# в цикле будем добавлять по 100 записей со смещением, потому что максимально можно запросить только 100
while len(posts) < post_count:
    url = 'https://api.vk.com/method/wall.get'

    params = {'domain':'skyeng',
          'count':100,
          'offset': off,
          'access_token': token,
          'v':'5.131'
        }
    result = requests.get(url, params = params).json()
    df = pd.DataFrame(result['response']['items'])
    posts = pd.concat([posts,df], ignore_index=True)
    print(f'Добавлены записи, длина списка - {len(posts)}')
    off+=100
    time.sleep(0.4)
else:
    print('Цикл окончен')

Добавлены записи, длина списка - 200
Добавлены записи, длина списка - 300
Добавлены записи, длина списка - 400
Добавлены записи, длина списка - 500
Добавлены записи, длина списка - 600
Добавлены записи, длина списка - 700
Добавлены записи, длина списка - 800
Добавлены записи, длина списка - 900
Добавлены записи, длина списка - 1000
Добавлены записи, длина списка - 1100
Добавлены записи, длина списка - 1200
Добавлены записи, длина списка - 1300
Добавлены записи, длина списка - 1400
Добавлены записи, длина списка - 1500
Добавлены записи, длина списка - 1600
Добавлены записи, длина списка - 1700
Добавлены записи, длина списка - 1800
Добавлены записи, длина списка - 1900
Добавлены записи, длина списка - 2000
Добавлены записи, длина списка - 2100
Добавлены записи, длина списка - 2200
Добавлены записи, длина списка - 2300
Добавлены записи, длина списка - 2400
Добавлены записи, длина списка - 2500
Добавлены записи, длина списка - 2600
Добавлены записи, длина списка - 2700
Добавлены записи, дл

In [6]:
print('Кол-во записей:', len(posts))

Кол-во записей: 8741


In [7]:
posts.head()

,donut,comments,marked_as_ads,short_text_rate,hash,type,carousel_offset,attachments,date,from_id,...,post_type,reposts,text,views,edited,copy_history,signer_id,zoom_text,marked_as_author_ad,geo
0,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,7r_VssrRO1adrWpGoTKQYfg0g8AU,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1697032513,-51890028,...,post,"{'count': 8, 'user_reposted': 0}","«Коллеги, нужен синк по лидам. Смэтчимся по сл...",{'count': 2500},NaN,NaN,NaN,NaN,NaN,NaN
1,{'is_donut': False},"{'can_post': 1, 'count': 3, 'groups_can_post':...",0,0.8,tyQUYUZB0ldV_E8P17HvuvpGtZfd,post,NaN,"[{'type': 'video', 'video': {'response_type': ...",1696936308,-51890028,...,post,"{'count': 2, 'user_reposted': 0}",,{'count': 5893},NaN,NaN,NaN,NaN,NaN,NaN
2,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,XRTTBZ1MMzIxfHQhRS4Ra7Eik7VR,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1696854134,-51890028,...,post,"{'count': 9, 'user_reposted': 0}","“I’ve got a ton of homework to do, so I better...",{'count': 4078},1.696856e+09,NaN,NaN,NaN,NaN,NaN
3,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,WA_mAhtoeV3EHjXioS3Z5Q89qLBx,post,NaN,"[{'type': 'video', 'video': {'response_type': ...",1696835541,-51890028,...,post,"{'count': 6, 'user_reposted': 0}",,{'count': 6695},NaN,NaN,NaN,NaN,NaN,NaN
4,{'is_donut': False},"{'can_post': 1, 'count': 2, 'groups_can_post':...",0,0.8,_Dh35ZbNBcLADpNXORfLH4k_hUFV,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1696755420,-51890028,...,post,"{'count': 6, 'user_reposted': 0}",Мыслить нужно позитивно и видеть за окном не г...,{'count': 4669},NaN,NaN,NaN,NaN,NaN,NaN


## Часть 4

__Выгружаем 50 самых популярных постов. Метрикой популярности мы считаем сумму лайков, репостов и комментариев__

In [8]:
# вытащи из словарей столбцов comments, likes, reposts только нужные значения

# Функция для подсчета значений
def get_count(x):
    try:
        return x['count']
    except:
        x.fillna(0, inplace = True)

# заполни даными с прошлого шага новые столбцы

posts['comments_count'] = posts['comments'].apply(get_count)

posts['likes_count'] = posts['likes'].apply(get_count)

posts['reposts_count'] = posts['likes'].apply(get_count)

In [9]:
# заполним данными столбец с суммой по всем активностям
posts['activity'] = posts['comments_count'] + posts['likes_count'] + posts['reposts_count']
posts[['comments_count', 'likes_count', 'reposts_count', 'activity']]

,comments_count,likes_count,reposts_count,activity
0,0,19,19,38
1,3,31,31,65
2,0,29,29,58
3,0,22,22,44
4,2,26,26,54
...,...,...,...,...
8736,0,2,2,4
8737,0,4,4,8
8738,4,4,4,12
8739,0,7,7,14


In [10]:
# выполним сортировку, чтобы выделить 50 лучших постов
posts.sort_values(by='activity', ascending=False, inplace = True)
posts.head(50)

,donut,comments,marked_as_ads,short_text_rate,hash,type,carousel_offset,attachments,date,from_id,...,edited,copy_history,signer_id,zoom_text,marked_as_author_ad,geo,comments_count,likes_count,reposts_count,activity
2587,{'is_donut': False},"{'can_post': 1, 'count': 239, 'groups_can_post...",0,0.8,sRDZPK49E-SLNnE88ltmFrqoJJPJ,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1554800244,-51890028,...,1.554800e+09,NaN,NaN,NaN,NaN,NaN,239,7053,7053,14345
1540,{'is_donut': False},"{'can_post': 1, 'count': 10962, 'groups_can_po...",0,0.8,aEuezpCgw_I8rzbQjeFS48gsq9N5,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1602590409,-51890028,...,NaN,NaN,NaN,NaN,NaN,NaN,10962,117,117,11196
8234,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,bfxAsmC-jJblCq_if0xcOcU9dMZv,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1416930109,-51890028,...,1.416991e+09,NaN,NaN,NaN,NaN,NaN,0,4013,4013,8026
2266,{'is_donut': False},"{'can_post': 1, 'count': 50, 'groups_can_post'...",0,0.8,_lW_eowfzsxImflvIu8cBp5n1xOK,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1568444249,-51890028,...,1.568449e+09,NaN,NaN,NaN,NaN,NaN,50,1944,1944,3938
501,{'is_donut': False},"{'can_post': 1, 'count': 22, 'groups_can_post'...",0,0.8,tYuRpV0gSnuFcYCxzJ29x1jPwR_0,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1662736193,-51890028,...,NaN,NaN,NaN,NaN,NaN,NaN,22,1887,1887,3796
2131,{'is_donut': False},"{'can_post': 1, 'count': 54, 'groups_can_post'...",0,0.8,8lDFzp4oAd-9t96Ym_XG82qlwPQR,post,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1574414757,-51890028,...,NaN,NaN,NaN,NaN,NaN,NaN,54,1629,1629,3312
1475,{'is_donut': False},"{'can_post': 1, 'count': 2891, 'groups_can_pos...",0,0.8,PQEyGAjUKF7x1Oxy87wwca-hc1Zz,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1604923204,-51890028,...,NaN,NaN,NaN,NaN,NaN,NaN,2891,134,134,3159
1825,{'is_donut': False},"{'can_post': 1, 'count': 84, 'groups_can_post'...",0,0.8,v25eNrcc0jbWVtNym6qGLJ5kx7v5,post,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1589373062,-51890028,...,NaN,NaN,NaN,NaN,NaN,NaN,84,1479,1479,3042
1691,{'is_donut': False},"{'can_post': 1, 'count': 26, 'groups_can_post'...",0,0.8,MYhuVcfmWP5kV894idtESm63sP21,post,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1596104809,-51890028,...,1.596128e+09,NaN,NaN,NaN,NaN,NaN,26,1464,1464,2954
1935,{'is_donut': False},"{'can_post': 1, 'count': 66, 'groups_can_post'...",0,0.8,kiZBYpS6pD70NxtQQsQyBJkubkxw,post,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1583935214,-51890028,...,NaN,NaN,NaN,NaN,NaN,NaN,66,1424,1424,2914


## Часть 5

__Почистим данные и сохраним результат в файл для отправки копирайтерам__


In [50]:
from datetime import datetime
# 
posts = posts[['date', 'text', 'likes_count', 'reposts_count', 'comments_count', 'activity']].head(50)
posts.reset_index(drop = False, inplace = True)
posts.head()

,index,date,text,likes_count,reposts_count,comments_count,activity
0,0,2019-04-09 11:57:24,Через 15 дней вы накачаете пресс и выучите анг...,7053,7053,239,14345
1,1,2020-10-13 15:00:09,КОНКУРС! Выиграйте уроки английского и бьюти-б...,117,117,10962,11196
2,2,2014-11-25 18:41:49,100 полезных английских фраз\nЭти фразы выруча...,4013,4013,0,8026
3,3,2019-09-14 09:57:29,😑Предлоги в английском — больная тема для всех...,1944,1944,50,3938
4,4,2022-09-09 18:09:53,8 сентября королева Елизавета II мирно скончал...,1887,1887,22,3796


In [62]:

# переведем из таймстемпа в нормальный формат даты
posts['date'] = posts['date'].apply(lambda x:datetime.fromtimestamp(x))
posts.to_csv('posts.csv', sep = ';' )

In [71]:
posts

,date,text,likes_count,reposts_count,comments_count,activity
0,2019-04-09 11:57:24,Через 15 дней вы накачаете пресс и выучите анг...,7053,7053,239,14345
1,2020-10-13 15:00:09,КОНКУРС! Выиграйте уроки английского и бьюти-б...,117,117,10962,11196
2,2014-11-25 18:41:49,100 полезных английских фраз\nЭти фразы выруча...,4013,4013,0,8026
3,2019-09-14 09:57:29,😑Предлоги в английском — больная тема для всех...,1944,1944,50,3938
4,2022-09-09 18:09:53,8 сентября королева Елизавета II мирно скончал...,1887,1887,22,3796
5,2019-11-22 12:25:57,"Кот Виктор показывает, как стал причиной сканд...",1629,1629,54,3312
6,2020-11-09 15:00:04,КОНКУРС! Выиграйте уроки английского и боксы с...,134,134,2891,3159
7,2020-05-13 15:31:02,Готовы ли вы к жизни в англоговорящем городе? ...,1479,1479,84,3042
8,2020-07-30 13:26:49,Подписывайтесь на [club51890028|Skyeng],1464,1464,26,2954
9,2020-03-11 17:00:14,,1424,1424,66,2914
